## Setup

In [2]:
import os
import pandas as pd
import math
import numpy as np
from statistics import mean
# import nltk
import torch
from transformers import BertTokenizer, BertForSequenceClassification

from transformacoes.pre_processamento import get_processed_df

c:\Users\Gabriel\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Pré-processamento do texto

In [14]:
true_path=r"dados\DataSet_Misinfo_TRUE.csv"
fake_path=r"dados\DataSet_Misinfo_FAKE.csv"

df_true = pd.read_csv(true_path, nrows=1000)
df_fake = pd.read_csv(fake_path, nrows=1000)

In [3]:
df_true.info()
df_true.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  100 non-null    int64 
 1   text        100 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


,Unnamed: 0,text
0,0,The head of a conservative Republican faction ...
1,1,Transgender people will be allowed for the fir...
2,2,The special counsel investigation of links bet...
3,3,Trump campaign adviser George Papadopoulos tol...
4,4,President Donald Trump called on the U.S. Post...


In [4]:
df_fake.info()
df_fake.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  100 non-null    int64 
 1   text        100 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


,Unnamed: 0,text
0,0,Donald Trump just couldn t wish all Americans ...
1,1,House Intelligence Committee Chairman Devin Nu...
2,2,"On Friday, it was revealed that former Milwauk..."
3,3,"On Christmas day, Donald Trump announced that ..."
4,4,Pope Francis used his annual Christmas Day mes...


In [15]:
df_true.drop('Unnamed: 0', axis=1, inplace=True)
df_fake.drop('Unnamed: 0', axis=1, inplace=True)

In [16]:
df_true['flag'] = 0
df_fake['flag'] = 1

In [17]:
df = pd.concat([df_true, df_fake], ignore_index=True)

In [18]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2000 non-null   object
 1   flag    2000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 31.4+ KB


,text,flag
0,The head of a conservative Republican faction ...,0
1,Transgender people will be allowed for the fir...,0
2,The special counsel investigation of links bet...,0
3,Trump campaign adviser George Papadopoulos tol...,0
4,President Donald Trump called on the U.S. Post...,0


In [37]:
df.groupby('flag').size()

flag
0    1000
1    1000
dtype: int64

In [36]:
df = df[~df['text'].isnull()]

In [42]:
df['text'] = df['text'].str.lower()

In [43]:
df['text']

0       the head of a conservative republican faction ...
1       transgender people will be allowed for the fir...
2       the special counsel investigation of links bet...
3       trump campaign adviser george papadopoulos tol...
4       president donald trump called on the u.s. post...
                              ...                        
1995    not long ago, rep. jason chaffetz (r-utah), sa...
1996    according to a new report, health insurance pr...
1997    the congressional budget office released their...
1998    donald trump was in a room full of irish repor...
1999    donald trump s war on the free press just ente...
Name: text, Length: 2000, dtype: object

In [2]:
# Rodar somente se o dataframe total nao tiver sido gerado
# get_processed_df(true_path=r"dados\DataSet_Misinfo_TRUE.csv", fake_path=r"dados\DataSet_Misinfo_FAKE.csv", final_path=os.path.join(os.path.split(true_path)[0], 'data.csv'))


Lendo os dados brutos

In [3]:
df = pd.read_csv(r"dados\data.csv")

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78588 entries, 0 to 78587
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    78588 non-null  object
 1   flag    78588 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.2+ MB


,text,flag
0,The head of a conservative Republican faction ...,1
1,Transgender people will be allowed for the fir...,1
2,The special counsel investigation of links bet...,1
3,Trump campaign adviser George Papadopoulos tol...,1
4,President Donald Trump called on the U.S. Post...,1


In [40]:
df['text'].str.lower()

0       the head of a conservative republican faction ...
1       transgender people will be allowed for the fir...
2       the special counsel investigation of links bet...
3       trump campaign adviser george papadopoulos tol...
4       president donald trump called on the u.s. post...
                              ...                        
1995    not long ago, rep. jason chaffetz (r-utah), sa...
1996    according to a new report, health insurance pr...
1997    the congressional budget office released their...
1998    donald trump was in a room full of irish repor...
1999    donald trump s war on the free press just ente...
Name: text, Length: 2000, dtype: object

In [22]:
print(df['text'][78612])

The USA wants to divide Syria.

Great Britain, France, Germany and other EU and NATO states are to send their own troops and, above all, money to the areas of the north-east of the Euphrates, according to the will of the USA. This should secure the US's strategy, the division of Syria along the Euphrates, in the long term even after a planned partial withdrawal of US troops from the area.

&nbsp;


Obtendo a distribuição de palavras de cada texto

In [10]:
df['text'][1]

'Transgender people will be allowed for the first time to enlist in the U.S. military starting on Monday as ordered by federal courts, the Pentagon said on Friday, after President Donald Trump’s administration decided not to appeal rulings that blocked his transgender ban. Two federal appeals courts, one in Washington and one in Virginia, last week rejected the administration’s request to put on hold orders by lower court judges requiring the military to begin accepting transgender recruits on Jan. 1. A Justice Department official said the administration will not challenge those rulings. “The Department of Defense has announced that it will be releasing an independent study of these issues in the coming weeks. So rather than litigate this interim appeal before that occurs, the administration has decided to wait for DOD’s study and will continue to defend the president’s lawful authority in District Court in the meantime,” the official said, speaking on condition of anonymity. In Septem

In [30]:
print(mean([len(i.split()) for i in df['text'].dropna()]))

12.268218397228122
540.8433455856856


In [35]:
df.groupby('flag').size()

label
0    35028
1    37106
dtype: int64

Importando o modelo pré-treinado

In [4]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Change num_labels according to your classification task

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [5]:
# Tokenize and preprocess input text
def preprocess_text(text):
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        truncation=True,
        padding='max_length',
        max_length=128,  # Adjust according to your input length requirements
        return_tensors='pt'
    )
    return inputs

# Example text for classification
text = "This is an example sentence for classification."

# Preprocess input text
inputs = preprocess_text(text)

# Perform classification
outputs = model(**inputs)

# Get predicted class label
predicted_label = torch.argmax(outputs.logits).item()
predicted_class = model.config.id2label[predicted_label]

print("Predicted Class:", predicted_class)

Predicted Class: LABEL_0


In [40]:
torch.argmax(outputs.logits).item()

0